# **BERT with RAG**

In [ ]:
# Installing offline dependencies
!pip install -U --no-deps /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -U --no-deps /kaggle/input/datasets-214/datasets-2.14.5-py3-none-any.whl
!pip install -U --no-deps /kaggle/input/optimum-113/optimum-1.13.2-py3-none-any.whl
!pip install -U --no-deps /kaggle/input/transformers-432/transformers-4.32.1-py3-none-any.whl

In [ ]:
import gc
from time import time
from concurrent.futures import ThreadPoolExecutor
import ctypes

import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# For RAG
import faiss
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_from_disk, Dataset

NUM_TITLES = 5
MAX_SEQ_LEN = 512
MODEL_PATH = "/kaggle/input/bge-small-faiss/"

# For LLM
from transformers import AutoTokenizer, AutoModel

MAX_LENGTH = 4096
MAX_CONTEXT = 1200

In [ ]:
df = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv", index_col="id")

## 1. Wikipedia Retrieval Augmented Generation (RAG)

The following code is adapted from [the notebook of @MGöksu](https://www.kaggle.com/code/mgoksu/0-807-sharing-my-trained-with-context-model) and [the notebook of @MB](https://www.kaggle.com/code/mbanaei/86-2-with-only-270k-articles/notebook). We use the [bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5) to embed the Wikipedia dataset.

In [ ]:
# New SentenceTransformer class similar to the one used in @Mgöksu notebook but relying on the transformers library only

class SentenceTransformer:
    def __init__(self, checkpoint, device="cuda:0"):
        self.device = device
        self.checkpoint = checkpoint
        self.model = AutoModel.from_pretrained(checkpoint).to(self.device).half()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def transform(self, batch):
        tokens = self.tokenizer(batch["text"], truncation=True, padding=True, return_tensors="pt", max_length=MAX_SEQ_LEN)
        return tokens.to(self.device)  

    def get_dataloader(self, sentences, batch_size=32):
        sentences = ["Represent this sentence for searching relevant passages: " + x for x in sentences]
        dataset = Dataset.from_dict({"text": sentences})
        dataset.set_transform(self.transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        return dataloader

    def encode(self, sentences, show_progress_bar=False, batch_size=32):
        dataloader = self.get_dataloader(sentences, batch_size=batch_size)
        pbar = tqdm(dataloader) if show_progress_bar else dataloader

        embeddings = []
        for batch in pbar:
            with torch.no_grad():
                e = self.model(**batch).pooler_output
                e = F.normalize(e, p=2, dim=1)
                embeddings.append(e.detach().cpu().numpy())
        embeddings = np.concatenate(embeddings, axis=0)
        return embeddings

In [ ]:
# Load embedding model
start = time()
print(f"Starting prompt embedding, t={time() - start :.1f}s")
model = SentenceTransformer(MODEL_PATH, device="cuda:0")

# Get embeddings of prompts
f = lambda row : " ".join([row["prompt"], row["A"], row["B"], row["C"], row["D"], row["E"]])
inputs = df.apply(f, axis=1).values # better results than prompt only
prompt_embeddings = model.encode(inputs, show_progress_bar=False)

# Search closest sentences in the wikipedia index 
print(f"Loading faiss index, t={time() - start :.1f}s")
faiss_index = faiss.read_index(MODEL_PATH + '/faiss.index')
# faiss_index = faiss.index_cpu_to_all_gpus(faiss_index) # causes OOM, and not that long on CPU

print(f"Starting text search, t={time() - start :.1f}s")
search_index = faiss_index.search(np.float32(prompt_embeddings), NUM_TITLES)[1]

print(f"Starting context extraction, t={time() - start :.1f}s")
dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
for i in range(len(df)):
    df.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# Free memory
faiss_index.reset()
del faiss_index, prompt_embeddings, model, dataset
# clean_memory()
print(f"Context added, t={time() - start :.1f}s")

In [ ]:
from sklearn.model_selection import train_test_split

features = ['prompt', 'A', 'B', 'C', 'D', 'E', 'context']

y = df['answer']
X = df[features]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [ ]:
from transformers import AutoTokenizer
model_dir = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [ ]:
options = 'ABCDE'
indices = list(range(5))

option_index = {option: index for option, index in zip(options, indices)}
index_option = {index: option for option, index in zip(options, indices)}
train_dataset = Dataset.from_pandas(df)

def preprocess(examples):
    text = examples['context'] + examples['prompt']
    first_sentences = [text] * 5
    second_sentences = []
    for option in options:
        second_sentences.append(examples[option])
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    tokenized_examples['label'] = option_index[examples['answer']]
    return tokenized_examples

tokenized_train_ds = train_dataset.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer', 'context'])

In [ ]:
# Following datacollator (adapted from https://huggingface.co/docs/transformers/tasks/multiple_choice)
# will dynamically pad our questions at batch-time so we don't have to make every question the length
# of our longest question.
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
model = AutoModelForMultipleChoice.from_pretrained(model_dir)

In [ ]:
model_dir = 'bert-base-context'
training_args = TrainingArguments(
    output_dir=model_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=15,
    weight_decay=0.01,
    report_to='none',
    logging_steps=50,
)

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_train_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
!pip install psutil GPUtil

In [ ]:
import threading
import psutil
import GPUtil
import time
import matplotlib.pyplot as plt

# Function to monitor CPU and GPU usage
def monitor_resources(stop_event, interval=1):
    cpu_usage = []
    gpu_usage = []
    timestamps = []

    while not stop_event.is_set():
        timestamps.append(time.time())
        cpu_usage.append(psutil.cpu_percent(interval=None))

        gpus = GPUtil.getGPUs()
        gpu_usage.append(gpus[0].load * 100 if gpus else 0)  # Assumes one GPU

        time.sleep(interval)

    plt.figure(figsize=(10, 6))
    plt.plot(timestamps, cpu_usage, label='CPU Usage (%)')
    plt.plot(timestamps, gpu_usage, label='GPU Usage (%)')
    plt.xlabel('Time (s)')
    plt.ylabel('Usage (%)')
    plt.title('CPU and GPU Usage Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    print(timestamps[-1] - timestamps[0])
    print(max(cpu_usage))
    print(max(gpu_usage))

# Monitoring setup
stop_event = threading.Event()
monitor_thread = threading.Thread(target=monitor_resources, args=(stop_event,))

# Start monitoring
monitor_thread.start()

# Start training
trainer.train()

# Stop monitoring
stop_event.set()
monitor_thread.join()

# At this point, the monitoring thread has completed
print("Training and monitoring completed.")